In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [7]:
# load .npy files from the work_dir/bnrelu/ folder
trues = np.load('../work_dirs/bnrelu/saved/trues.npy')
preds = np.load('../work_dirs/bnrelu/saved/preds.npy')

In [18]:
# get pixel wise mse 
mse = np.mean((trues - preds)**2, axis=0)
# Get total mse averaged over all pixels
total_mse = np.mean(mse)
print (total_mse)


78.51834


In [21]:
mse[0,0,52,52]

841.3663

In [55]:
# load npy files from the work_dir/7daysv2_unet/saved folder
trues1 = np.load('../work_dirs/7daysv2_unet/saved/trues.npy')
preds1 = np.load('../work_dirs/7daysv2_unet/saved/preds.npy')

mse1 = np.mean((trues1 - preds1)**2, axis=0)
total_mse1 = np.mean(mse1)
print (total_mse1)

78.124756


In [25]:
print (mse1[0,0,52,52])

903.91724


In [56]:
# sum over first axis in preds1 and trues1 and divide by 240
preds1 = np.sum(preds1, axis=0)
trues1 = np.sum(trues1, axis=0)
preds1 = preds1 / 240
trues1 = trues1 / 240

In [31]:
preds1.shape

(1, 1, 128, 128)

In [34]:
mse1.shape

(1, 1, 128, 128)

In [48]:
%matplotlib qt5

In [60]:
mse1[52,52]

904.0

In [57]:
# round values to nearest integer in mse1
mse1 = np.round(mse1[0,0])


In [61]:
mse  = np.round(mse[0,0])

In [75]:
import numpy as np
import matplotlib.pyplot as plt

# Generate a random binary image (128x128)
image = preds1[0,0]

# Calculate the figure size based on the image size
fig_width = max(16, image.shape[1] * 0.5)
fig_height = max(12, image.shape[0] * 0.5)

# Create a figure with adjusted size
fig, ax = plt.subplots(figsize=(fig_width, fig_height))

color = np.where(mse > mse1, 'red', np.where(mse < mse1, 'blue', 'green'))
# Display the image with labeled pixels
im = ax.imshow(image, cmap='gray')
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        if mse1[i, j] > 0 or mse[i, j] > 0:
            ax.text(j, i, str(int(image[i,j]))+','+str(mse[i, j])+","+str(mse1[i,j]),
                    ha='center', va='center', color=color[i,j], fontsize=10)

# Add colorbar
cbar = fig.colorbar(im)

# Set labels and title
ax.set_xlabel('Columns')
ax.set_ylabel('Rows')
ax.set_title('Pixel Value Visualization')

# Show the plot
plt.show()


ValueError: array([['green', 'green', 'green', ..., 'red', 'blue', 'red'],
       ['green', 'green', 'green', ..., 'red', 'blue', 'green'],
       ['green', 'green', 'green', ..., 'red', 'red', 'red'],
       ...,
       ['blue', 'green', 'green', ..., 'green', 'green', 'green'],
       ['blue', 'green', 'green', ..., 'red', 'green', 'red'],
       ['red', 'green', 'blue', ..., 'green', 'red', 'red']], dtype='<U5') is not a valid value for color

In [68]:
mse.shape

(128, 128)

In [70]:
str(mse[i, j])+","+str(mse1[i,j])

'13.0,3.0'